In [16]:
!pip install cvxpy

    100% |████████████████████████████████| 921kB 11.6MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 11.3MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 18.4MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 17.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 5.8MB/s eta 0:00:01
    100% |████████████████████████████████| 17.3MB 64kB/s eta 0:00:011    79% |█████████████████████████▍      | 13.7MB 7.3MB/s eta 0:00:01    91% |█████████████████████████████▍  | 15.9MB 7.3MB/s eta 0:00:01
  Stored in directory: /home/nbuser/.cache/pip/wheels/5c/b7/db/a45957512b8e6191ac26315d3847917321775b3d1b4aeceb35
  Stored in directory: /home/nbuser/.cache/pip/wheels/94/e2/a6/64db723051c54017c248ea5a26e7f1459c0242d735a496dd55
  Stored in directory: /home/nbuser/.cache/pip/wheels/3a/ed/51/77c833462c3e757ce50c4b2b68bdf53f5d1745542fe567d740
Successfully built cvxpy scs multiprocess
scikit-bio 0.5.4 has requirement numpy<1.14.0,>=1.9.2, bu

In [17]:
import pandas as pd
import numpy as np
import cvxpy as cp

In [2]:
df = pd.read_csv('Data/DKSalaries Match Play 20190326.csv')
df.head()

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter
0,Rory McIlroy (12300482),Rory McIlroy,11700,42.8,5.2,4,NaN
1,Dustin Johnson (12300483),Dustin Johnson,11400,40.8,5.4,1,NaN
2,Justin Thomas (12300484),Justin Thomas,11000,37.7,3.8,5,NaN
3,Jon Rahm (12300485),Jon Rahm,10600,37.4,3.5,8,NaN
4,Brooks Koepka (12300486),Brooks Koepka,10300,33.3,2.1,3,NaN


In [3]:
quarters = {1:1,16:1,9:2,8:2,
          5:3,12:3,13:4,4:4,
          2:5,15:5,10:6,7:6,
          6:7,11:7,14:8,3:8}
df['Quarter'] = df.Group.map(quarters)
df.head()

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter
0,Rory McIlroy (12300482),Rory McIlroy,11700,42.8,5.2,4,4
1,Dustin Johnson (12300483),Dustin Johnson,11400,40.8,5.4,1,1
2,Justin Thomas (12300484),Justin Thomas,11000,37.7,3.8,5,3
3,Jon Rahm (12300485),Jon Rahm,10600,37.4,3.5,8,2
4,Brooks Koepka (12300486),Brooks Koepka,10300,33.3,2.1,3,8


In [4]:
df['Semi'] = (df.Quarter-1) // 2 + 1
df.head()

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter,Semi
0,Rory McIlroy (12300482),Rory McIlroy,11700,42.8,5.2,4,4,2
1,Dustin Johnson (12300483),Dustin Johnson,11400,40.8,5.4,1,1,1
2,Justin Thomas (12300484),Justin Thomas,11000,37.7,3.8,5,3,2
3,Jon Rahm (12300485),Jon Rahm,10600,37.4,3.5,8,2,1
4,Brooks Koepka (12300486),Brooks Koepka,10300,33.3,2.1,3,8,4


In [5]:
df[df.Name.isin(['Webb Simpson','Patrick Cantlay','Tony Finau','Sergio Garcia','Xander Schauffele'])]

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter,Semi
9,Xander Schauffele (12300491),Xander Schauffele,9400,33.7,2.7,9,2,1
13,Tony Finau (12300495),Tony Finau,9000,34.7,3.0,14,8,4
18,Sergio Garcia (12300500),Sergio Garcia,8500,33.9,2.1,16,1,1
19,Patrick Cantlay (12300501),Patrick Cantlay,8400,31.1,2.3,13,4,2
20,Webb Simpson (12300502),Webb Simpson,8300,34.4,2.2,7,6,3


In [6]:
#Add one column for each Quarter, and one column for each Semi
df_quarters = pd.get_dummies(df['Quarter']).add_prefix('Quarter_')
df_quarters.head()

,Quarter_1,Quarter_2,Quarter_3,Quarter_4,Quarter_5,Quarter_6,Quarter_7,Quarter_8
0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1


In [7]:
df = pd.merge(df,df_quarters,left_index=True,right_index=True)
df.head()

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter,Semi,Quarter_1,Quarter_2,Quarter_3,Quarter_4,Quarter_5,Quarter_6,Quarter_7,Quarter_8
0,Rory McIlroy (12300482),Rory McIlroy,11700,42.8,5.2,4,4,2,0,0,0,1,0,0,0,0
1,Dustin Johnson (12300483),Dustin Johnson,11400,40.8,5.4,1,1,1,1,0,0,0,0,0,0,0
2,Justin Thomas (12300484),Justin Thomas,11000,37.7,3.8,5,3,2,0,0,1,0,0,0,0,0
3,Jon Rahm (12300485),Jon Rahm,10600,37.4,3.5,8,2,1,0,1,0,0,0,0,0,0
4,Brooks Koepka (12300486),Brooks Koepka,10300,33.3,2.1,3,8,4,0,0,0,0,0,0,0,1


In [8]:
#Add semis as well
df_semis = pd.get_dummies(df.Semi).add_prefix('Semi_')
df_semis.head()

,Semi_1,Semi_2,Semi_3,Semi_4
0,0,1,0,0
1,1,0,0,0
2,0,1,0,0
3,1,0,0,0
4,0,0,0,1


In [9]:
df = pd.merge(df,df_semis,left_index=True,right_index=True)
df.head()

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter,Semi,Quarter_1,Quarter_2,Quarter_3,Quarter_4,Quarter_5,Quarter_6,Quarter_7,Quarter_8,Semi_1,Semi_2,Semi_3,Semi_4
0,Rory McIlroy (12300482),Rory McIlroy,11700,42.8,5.2,4,4,2,0,0,0,1,0,0,0,0,0,1,0,0
1,Dustin Johnson (12300483),Dustin Johnson,11400,40.8,5.4,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
2,Justin Thomas (12300484),Justin Thomas,11000,37.7,3.8,5,3,2,0,0,1,0,0,0,0,0,0,1,0,0
3,Jon Rahm (12300485),Jon Rahm,10600,37.4,3.5,8,2,1,0,1,0,0,0,0,0,0,1,0,0,0
4,Brooks Koepka (12300486),Brooks Koepka,10300,33.3,2.1,3,8,4,0,0,0,0,0,0,0,1,0,0,0,1


### Optimize

In [18]:
sal = df.Salary.values
W = df['Win Group'].values
q1 = df.Quarter_1
q2 = df.Quarter_2
q3 = df.Quarter_3
q4 = df.Quarter_4
q5 = df.Quarter_5
q6 = df.Quarter_6
q7 = df.Quarter_7
q8 = df.Quarter_8
s1 = df.Semi_1
s2 = df.Semi_2
s3 = df.Semi_3
s4 = df.Semi_4

n = len(df)
x = cp.Variable(n,boolean=True)

#Contraints
#1. total sal under cap
#2. use exactly 6
#3. no 2 from same quarter
#4. at least one from every semi

penalty = 0
cap = 50000
constraints = [cp.sum(x) == 6,
             x*sal <= cap,
             x*q1 <= 1,x*q2 <= 1,x*q3 <= 1,x*q4 <= 1,
              x*q5 <= 1,x*q6 <= 1,x*q7 <= 1,x*q8 <= 1,
              x*s1 >= 1, x*s2 >= 1, x*s3 >= 1, x*s4 >= 1]

# Form objective.
obj = cp.Maximize(x*W)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal objective value", prob.value)
opt_team = df['Name + ID'][x.value>.5].tolist()
print(opt_team)

status: optimal
optimal objective value 194.0000000046871
['Xander Schauffele (12300491)', 'Tony Finau (12300495)', 'Sergio Garcia (12300500)', 'Patrick Cantlay (12300501)', 'Webb Simpson (12300502)', 'Byeong-Hun An (12300538)']


In [19]:
#Start a df with teams
df_teams = pd.DataFrame(columns=['p1','p2','p3','p4','p5','p6'])
df_teams.loc[0] = opt_team
df_teams

,p1,p2,p3,p4,p5,p6
0,Xander Schauffele (12300491),Tony Finau (12300495),Sergio Garcia (12300500),Patrick Cantlay (12300501),Webb Simpson (12300502),Byeong-Hun An (12300538)


In [20]:
#Remove each one (run immediately after cell above)

for player in opt_team:
    df2 = df[df['Name + ID'] != player]
    sal = df2.Salary.values
    W = df2['Win Group'].values
    q1 = df2.Quarter_1
    q2 = df2.Quarter_2
    q3 = df2.Quarter_3
    q4 = df2.Quarter_4
    q5 = df2.Quarter_5
    q6 = df2.Quarter_6
    q7 = df2.Quarter_7
    q8 = df2.Quarter_8
    s1 = df2.Semi_1
    s2 = df2.Semi_2
    s3 = df2.Semi_3
    s4 = df2.Semi_4

    n = len(df2)
    x = cp.Variable(n,boolean=True)

    #Contraints
    #1. total sal under cap
    #2. use exactly 6
    #3. no 2 from same quarter
    #4. at least one from every semi

    penalty = 0
    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*sal <= cap,
                 x*q1 <= 1,x*q2 <= 1,x*q3 <= 1,x*q4 <= 1,
                  x*q5 <= 1,x*q6 <= 1,x*q7 <= 1,x*q8 <= 1,
                  x*s1 >= 1, x*s2 >= 1, x*s3 >= 1, x*s4 >= 1]

    # Form objective.
    obj = cp.Maximize(x*W)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    #print("status:", prob.status)
    #print("optimal objective value", prob.value)
    opt_team = df2['Name + ID'][x.value>.5].tolist()
    #print('No {} (obj = {:.1f}): '.format(player,prob.value),opt_team)
    df_teams.loc[len(df_teams)] = opt_team
    
df_teams

,p1,p2,p3,p4,p5,p6
0,Xander Schauffele (12300491),Tony Finau (12300495),Sergio Garcia (12300500),Patrick Cantlay (12300501),Webb Simpson (12300502),Byeong-Hun An (12300538)
1,Rory McIlroy (12300482),Sergio Garcia (12300500),Webb Simpson (12300502),Gary Woodland (12300508),Hao-Tong Li (12300528),Byeong-Hun An (12300538)
2,Rory McIlroy (12300482),Sergio Garcia (12300500),Webb Simpson (12300502),Gary Woodland (12300508),Hao-Tong Li (12300528),Byeong-Hun An (12300538)
3,Dustin Johnson (12300483),Tony Finau (12300495),Webb Simpson (12300502),Gary Woodland (12300508),Keegan Bradley (12300520),Byeong-Hun An (12300538)
4,Rory McIlroy (12300482),Sergio Garcia (12300500),Webb Simpson (12300502),Gary Woodland (12300508),Hao-Tong Li (12300528),Byeong-Hun An (12300538)
5,Rory McIlroy (12300482),Tony Finau (12300495),Sergio Garcia (12300500),Gary Woodland (12300508),Thorbjorn Olesen (12300533),Byeong-Hun An (12300538)
6,Tiger Woods (12300493),Tony Finau (12300495),Sergio Garcia (12300500),Webb Simpson (12300502),Gary Woodland (12300508),Keegan Bradley (12300520)


In [21]:
#Randomly add more teams

#First add value field to serve as proxy for selection probability
df['value'] = df['Win Group'] / df.Salary * 1000
df.sort_values(by='value',ascending=False).head()

,Name + ID,Name,Salary,Win Group,Win Event,Group,Quarter,Semi,Quarter_1,Quarter_2,...,Quarter_4,Quarter_5,Quarter_6,Quarter_7,Quarter_8,Semi_1,Semi_2,Semi_3,Semi_4,value
56,Byeong-Hun An (12300538),Byeong-Hun An,6300,26.2,1.5,11,7,4,0,0,...,0,0,0,1,0,0,0,0,1,4.158730
20,Webb Simpson (12300502),Webb Simpson,8300,34.4,2.2,7,6,3,0,0,...,0,0,1,0,0,0,0,1,0,4.144578
18,Sergio Garcia (12300500),Sergio Garcia,8500,33.9,2.1,16,1,1,1,0,...,0,0,0,0,0,1,0,0,0,3.988235
13,Tony Finau (12300495),Tony Finau,9000,34.7,3.0,14,8,4,0,0,...,0,0,0,0,1,0,0,0,1,3.855556
46,Hao-Tong Li (12300528),Hao-Tong Li,6800,26.0,1.2,3,8,4,0,0,...,0,0,0,0,1,0,0,0,1,3.823529


In [22]:
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    #print(len(L),len(probs))
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

In [23]:
p = pick_element_per_probs(df['Name + ID'],df.value)
p

'Marc Leishman (12300497)'

In [24]:
df['Quarter'][df['Name + ID']==p].iloc[0]

7

In [25]:
a = {1,2,3}
all_semis = {1,2,3,4}
list(all_semis - a)[0]

4

In [106]:
cap = 200 #add this many more teams to 7 above
num_teams = 0
df_teams = pd.DataFrame(columns=['p1','p2','p3','p4','p5','p6'])
df_all_teams = df_teams.copy()
while num_teams < cap:
    #print('New Team')
    df_now = df.copy()
    curr_team = []
    semis_chosen = set([])
    all_semis = {1,2,3,4}
    for k in range(6):
        #If we're at the last and still have an empty semi, pick player from there
        if ((k==5) & (len(semis_chosen) < 4)):
            semi_left = list(all_semis - semis_chosen)[0]
            df_now = df_now[df_now.Semi == semi_left]
            #print('One semi left: {}'.format(semi_left))
            
        player = pick_element_per_probs(df_now['Name + ID'].tolist(),df_now.value.tolist())
        curr_team.append(player)
        #Remove anyone from the same quarter
        curr_qtr = df_now['Quarter'][df_now['Name + ID']==player].iloc[0]
        curr_semi = df_now['Semi'][df_now['Name + ID']==player].iloc[0]
        semis_chosen = semis_chosen.union({curr_semi})
        df_now = df_now[df_now['Quarter']!= curr_qtr]
        #print('  Quarter {}, Semi {}'.format(curr_qtr,curr_semi))
    #Now see if team has right salary
    df_this_team = df[df['Name + ID'].isin(curr_team)].sort_values(by='Salary',ascending=False)
    curr_team = df_this_team['Name + ID'].tolist()
    curr_sal = df_this_team.Salary.sum()
    if ((curr_sal <= 50000) & (curr_sal >= 48000)):
        #make sure team is unique
        #print(curr_team,curr_sal)
        df_all_teams.loc[len(df_all_teams)] = curr_team
        num_teams += 1
        
#df_all_teams.head(10)

In [107]:

#df_all_teams.to_csv('Lineups/match_play_lineups.csv')

# See how teams are doing

In [108]:
# df_teams= pd.read_csv('Lineups/match_play_lineups.csv')
# df_teams = df_teams[['p' + str(i) for i in np.arange(1,7)]]
# df_teams.head()

df_teams = df_all_teams.copy()
#df_teams.head()

In [109]:
players_left = ['Bjerr',
                'Tiger',
                'Kuchar',
                'Sergio',
               'Kisner',
                'Molinari',
                'Kevin Na',
                'Oosth'
               ]
full_names_left = []
for p in players_left:
    if len (df[df.Name.str.contains(p)]) > 1:
        print('Multiple matches for {}'.format(p))
    full_names_left.append(df[df.Name.str.contains(p)].iloc[0]['Name + ID'])
    
#print(full_names_left)

In [110]:
df_teams['Total'] = 0
df_teams['Semi1'] = 0
df_teams['Semi2'] = 0
df_teams['Semi3'] = 0
df_teams['Semi4'] = 0
#df_teams.head()

In [111]:
#Add number left from each semi to teams
def add_remaining(row):
    T = 0
    semis = [0,0,0,0]
    for c in df_teams.columns:
        p = row[c] #current player
        #see if he's still alive
        if p in full_names_left:
            T += 1
            #Find which semi
            curr_semi = df[df['Name + ID']==p].iloc[0]['Semi'] #should be 1, 2, 3, 4
            semis[curr_semi-1] += 1
    total_semis = 0
    for i, s in enumerate(semis):
        row['Semi' + str(i+1)] = s
        total_semis += int(s>0)
    row['Total Semis'] = total_semis
    row['Total'] = T
    return row
df_teams = df_teams.apply(add_remaining,axis=1)
#df_teams.head()

In [112]:
#Sort and view!
df_teams.sort_values(by=['Total Semis','Total'],ascending=[False,False],inplace=True)
df_teams[df_teams.Total >= 3]

,p1,p2,p3,p4,p5,p6,Total,Semi1,Semi2,Semi3,Semi4,Total Semis
16,Dustin Johnson (12300483),Bryson DeChambeau (12300490),Matt Kuchar (12300504),Keegan Bradley (12300520),Kevin Kisner (12300523),Kevin Na (12300537),3,1,0,1,1,3
84,Bryson DeChambeau (12300490),Tiger Woods (12300493),Matt Kuchar (12300504),Charles Howell (12300516),Keegan Bradley (12300520),Kevin Kisner (12300523),3,1,1,0,1,3
125,Tommy Fleetwood (12300489),Xander Schauffele (12300491),Tiger Woods (12300493),Jim Furyk (12300519),Kevin Kisner (12300523),Kevin Na (12300537),3,0,1,1,1,3
126,Justin Rose (12300488),Francesco Molinari (12300492),Sergio Garcia (12300500),Louis Oosthuizen (12300513),J.B. Holmes (12300522),Justin Harding (12300539),3,1,0,1,1,3
140,Justin Rose (12300488),Francesco Molinari (12300492),Tiger Woods (12300493),Henrik Stenson (12300512),Kevin Kisner (12300523),Lee Westwood (12300531),3,0,1,1,1,3
57,Rory McIlroy (12300482),Sergio Garcia (12300500),Webb Simpson (12300502),Matt Kuchar (12300504),Hao-Tong Li (12300528),Lucas Bjerregaard (12300540),3,2,1,0,0,2
161,Brooks Koepka (12300486),Justin Rose (12300488),Sergio Garcia (12300500),Matt Kuchar (12300504),Abraham Ancer (12300535),Lucas Bjerregaard (12300540),3,2,1,0,0,2
